## Ejercicio 2: Diseño y construcción de modelo data warehouse. (4pts)
**Descripción**: Se plantea diseñar un modelo data warehouse sencillo a partir del resultado del ejercicio 1, construido directamente con notebooks jupyter, para ello el modelo debe satisfacer las siguientes necesidades:
- Productos más vendidos
- Clientes con el mayor número de pedidos
- Corresponsal con el mayor número de pedidos
- Total de pagos diario y mensual por productos
- Total de pagos diario y mensual por clientes
- Total de pagos diario y mensual por corresponsal

**Requisitos:**
- Diseñar un modelo de data warehouse de tipo estrella, incluir tablas de hechos y dimensiones que satisfagan los reportes de BI
- Construcción de diagrama data warehouse.

**Entregables:**
Incluir dentro del proyecto:
- Diagrama de data warehouse (1p)
- Notebook Jupyter que realice lo siguiente:
    - Limpieza de datos: (2p)
        - Nombres en mayúsculas, sin tildes, sin ñ, sin guiones bajos y medios.
        - Valores numéricos con solo dos decimales redondeados al inmediato superior.
    - Integración de los datos: (1p)
        - Scripts con la creación de las tablas de dimensiones
        - Scripts con la creación de las tablas de hechos

In [1]:
!pip install sqlalchemy psycopg2-binary pandas

     |████████████████████████████████| 3.1 MB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 31.8 MB/s eta 0:00:01
     |████████████████████████████████| 579 kB 17.6 MB/s eta 0:00:01


In [27]:
#Conexión a la base de datos de postgres

from sqlalchemy import create_engine
import pandas as pd

usuario = 'postgres'       
contrasena = 'postgres'  
host = 'localhost'         
puerto = '5432'           
bd = 'mp3_db'   

#Credenciales para el datawerehouse        
bd_dw = 'postgres'   
#

conexion_dw = f'postgresql+psycopg2://{usuario}:{contrasena}@{host}:{puerto}/{bd_dw}'

conexion = f'postgresql+psycopg2://{usuario}:{contrasena}@{host}:{puerto}/{bd}'

engine = create_engine(conexion)
engine_dw=create_engine(conexion_dw)

#Verificación de la conexión
df_clientes = pd.read_sql('SELECT * FROM Clientes', engine)
df_clientes

,id_cliente,nombre,email,telefono,direccion,fecha_registro
0,1,Andrés Pérez,andres_perez@gmail.com,0987654321,Calle Olmedo 123,2025-04-29 17:08:02.812130
1,2,María-Luisa Ñáñez,maria_luisa@gmail.com,0991234567,Av. América 45-67,2025-04-29 17:08:02.812130
2,3,Juan Rodríguez,juan.rodriguez@hotmail.com,0987651234,Av. Quito 01-12,2025-04-29 17:08:02.812130
3,4,Carla Gómez,carla_gomez@yahoo.com,0981234567,Calle Bolívar 45-78,2025-04-29 17:08:02.812130
4,5,José Martínez,jose.martinez@gmail.com,0998765432,Calle Juan Montalvo 12-34,2025-04-29 17:08:02.812130
5,6,Ana-Lucia Ñiquiz,ana_niquiz@gmail.com,0984321786,Calle del Sol 23-67,2025-04-29 17:08:02.812130
6,7,Luis Torres,luis.torres@gmail.com,0997651234,Av. Loja 45-89,2025-04-29 17:08:02.812130
7,8,Mónica Fernández,monica_fernandez@gmail.com,0986453210,Barrio Los Andes,2025-04-29 17:08:02.812130
8,9,Pedro García,pedro_garcia@yahoo.com,0989876543,Calle Villavicencio,2025-04-29 17:08:02.812130
9,10,Gabriela Núñez,gabriela_nunez@hotmail.com,0994321678,Calle Mercurio 67-45,2025-04-29 17:08:02.812130


## *Limpieza de datos: (2p)*

- Nombres en mayúsculas, sin tildes, sin ñ, sin guiones bajos y medios.
- Valores numéricos con solo dos decimales redondeados al inmediato superior.

In [3]:
#Importación de librerias

import unicodedata
import numpy as np

In [4]:
# Limpieza de texto
def limpiar_texto(texto):
    if pd.isnull(texto): return texto
    texto = texto.upper().replace("_", " ").replace("-", " ")
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    texto = texto.replace('Ñ', 'N')
    return texto

In [5]:
# Limpieza a Clientes
df_clientes = pd.read_sql('SELECT * FROM Clientes', engine)
df_clientes['nombre'] = df_clientes['nombre'].apply(limpiar_texto)
df_clientes['email'] = df_clientes['email'].apply(limpiar_texto)
df_clientes

,id_cliente,nombre,email,telefono,direccion,fecha_registro
0,1,ANDRES PEREZ,ANDRES PEREZ@GMAIL.COM,0987654321,Calle Olmedo 123,2025-04-29 17:08:02.812130
1,2,MARIA LUISA NANEZ,MARIA LUISA@GMAIL.COM,0991234567,Av. América 45-67,2025-04-29 17:08:02.812130
2,3,JUAN RODRIGUEZ,JUAN.RODRIGUEZ@HOTMAIL.COM,0987651234,Av. Quito 01-12,2025-04-29 17:08:02.812130
3,4,CARLA GOMEZ,CARLA GOMEZ@YAHOO.COM,0981234567,Calle Bolívar 45-78,2025-04-29 17:08:02.812130
4,5,JOSE MARTINEZ,JOSE.MARTINEZ@GMAIL.COM,0998765432,Calle Juan Montalvo 12-34,2025-04-29 17:08:02.812130
5,6,ANA LUCIA NIQUIZ,ANA NIQUIZ@GMAIL.COM,0984321786,Calle del Sol 23-67,2025-04-29 17:08:02.812130
6,7,LUIS TORRES,LUIS.TORRES@GMAIL.COM,0997651234,Av. Loja 45-89,2025-04-29 17:08:02.812130
7,8,MONICA FERNANDEZ,MONICA FERNANDEZ@GMAIL.COM,0986453210,Barrio Los Andes,2025-04-29 17:08:02.812130
8,9,PEDRO GARCIA,PEDRO GARCIA@YAHOO.COM,0989876543,Calle Villavicencio,2025-04-29 17:08:02.812130
9,10,GABRIELA NUNEZ,GABRIELA NUNEZ@HOTMAIL.COM,0994321678,Calle Mercurio 67-45,2025-04-29 17:08:02.812130


In [6]:
# Limpieza a Productos
df_productos = pd.read_sql('SELECT * FROM Productos', engine)
df_productos['nombre'] = df_productos['nombre'].apply(limpiar_texto)
df_productos['descripcion'] = df_productos['descripcion'].apply(limpiar_texto)
df_productos

,id_producto,nombre,descripcion,precio,stock
0,1,TELEFONO XIAOMI,SMARTPHONE CON PANTALLA AMOLED,399.99001,50
1,2,AURICULARES N TECH,AURICULARES CON CANCELACION DE RUIDO,59.95001,100
2,3,LAPTOP HP,COMPUTADORA PORTATIL CON 16GB RAM,999.00040,25
3,4,TECLADO MECANICO,TECLADO RGB RETROILUMINADO,75.50004,80
4,5,MONITOR SAMSUNG,MONITOR LED 24 PULGADAS,180.45050,30
5,6,MOUSE GAMER,MOUSE OPTICO CON DPI AJUSTABLE,25.99005,200
6,7,SILLA ERGONOMICA,SILLA DE OFICINA AJUSTABLE,250.25005,15
7,8,CABLE HDMI,CABLE HDMI DE 2 METROS,12.50005,150
8,9,ROUTER WI FI,ROUTER DE ALTA VELOCIDAD,89.99005,50
9,10,DISCO DURO EXTERNO,DISCO DURO DE 1TB,120.75005,45


In [9]:
# Redondear valores numéricos de precio a dos decimales
df_productos['precio'] = np.ceil(df_productos['precio'] * 100) / 100
df_productos

,id_producto,nombre,descripcion,precio,stock
0,1,TELEFONO XIAOMI,SMARTPHONE CON PANTALLA AMOLED,400.00,50
1,2,AURICULARES N TECH,AURICULARES CON CANCELACION DE RUIDO,59.96,100
2,3,LAPTOP HP,COMPUTADORA PORTATIL CON 16GB RAM,999.01,25
3,4,TECLADO MECANICO,TECLADO RGB RETROILUMINADO,75.52,80
4,5,MONITOR SAMSUNG,MONITOR LED 24 PULGADAS,180.46,30
5,6,MOUSE GAMER,MOUSE OPTICO CON DPI AJUSTABLE,26.00,200
6,7,SILLA ERGONOMICA,SILLA DE OFICINA AJUSTABLE,250.26,15
7,8,CABLE HDMI,CABLE HDMI DE 2 METROS,12.51,150
8,9,ROUTER WI FI,ROUTER DE ALTA VELOCIDAD,90.00,50
9,10,DISCO DURO EXTERNO,DISCO DURO DE 1TB,120.76,45


In [11]:
# Se realzia tambien para Detalles_Orden y Ordenes
df_detalles = pd.read_sql('SELECT * FROM Detalles_Orden', engine)
df_ordenes = pd.read_sql('SELECT * FROM Ordenes', engine)

df_detalles

,id_detalle,id_orden,id_producto,cantidad,precio_unitario,subtotal
0,1,1,1,1,399.99001,399.99001
1,2,1,2,1,59.95001,59.95001
2,3,2,2,2,59.95002,119.90002
3,4,3,3,1,999.00004,999.00004
4,5,3,4,1,75.50004,75.50004
5,6,4,5,2,180.45005,360.90005
6,7,5,6,4,25.99005,103.96005
7,8,6,7,1,250.25005,250.25005
8,9,7,8,2,12.50005,25.00005
9,10,8,9,1,89.99005,89.99005


In [12]:
df_ordenes

,id_orden,id_cliente,fecha_orden,total
0,1,1,2025-04-29 17:08:02.819424,459.94002
1,2,2,2025-04-29 17:08:02.819424,119.90002
2,3,3,2025-04-29 17:08:02.819424,175.99000
3,4,4,2025-04-29 17:08:02.819424,1250.25008
4,5,5,2025-04-29 17:08:02.819424,89.99005
5,6,6,2025-04-29 17:08:02.819424,75.50005
6,7,7,2025-04-29 17:08:02.819424,220.45005
7,8,8,2025-04-29 17:08:02.819424,399.99005
8,9,9,2025-04-29 17:08:02.819424,180.45005
9,10,10,2025-04-29 17:08:02.819424,250.25005


In [15]:
for col in ['precio_unitario', 'subtotal']:
    df_detalles[col] = np.ceil(df_detalles[col] * 100) / 100

df_ordenes['total'] = np.ceil(df_ordenes['total'] * 100) / 100

df_detalles

,id_detalle,id_orden,id_producto,cantidad,precio_unitario,subtotal
0,1,1,1,1,400.00,400.00
1,2,1,2,1,59.96,59.96
2,3,2,2,2,59.96,119.91
3,4,3,3,1,999.01,999.01
4,5,3,4,1,75.52,75.52
5,6,4,5,2,180.46,360.91
6,7,5,6,4,26.00,103.97
7,8,6,7,1,250.26,250.26
8,9,7,8,2,12.51,25.01
9,10,8,9,1,90.00,90.00


In [16]:
df_ordenes

,id_orden,id_cliente,fecha_orden,total
0,1,1,2025-04-29 17:08:02.819424,459.95
1,2,2,2025-04-29 17:08:02.819424,119.91
2,3,3,2025-04-29 17:08:02.819424,175.99
3,4,4,2025-04-29 17:08:02.819424,1250.26
4,5,5,2025-04-29 17:08:02.819424,90.00
5,6,6,2025-04-29 17:08:02.819424,75.52
6,7,7,2025-04-29 17:08:02.819424,220.46
7,8,8,2025-04-29 17:08:02.819424,400.00
8,9,9,2025-04-29 17:08:02.819424,180.46
9,10,10,2025-04-29 17:08:02.819424,250.26


## **Creación de la base de datos para el DatawereHouse**

In [26]:
with engine_dw.connect() as conn:
    conn.execute(text('DROP TABLE IF EXISTS fact_orden, dim_tiempo, dim_producto, dim_cliente CASCADE;'))

    conn.execute(text('''
        CREATE TABLE dim_cliente (
            id_cliente INT PRIMARY KEY,
            nombre TEXT,
            email TEXT
        );
    '''))

    conn.execute(text('''
        CREATE TABLE dim_producto (
            id_producto INT PRIMARY KEY,
            nombre varchar,
            descripcion TEXT,
            precio NUMERIC(10, 2)
        );
    '''))

    conn.execute(text('''
        CREATE TABLE dim_tiempo (
            id_tiempo SERIAL PRIMARY KEY,
            fecha DATE,
            anio INT,
            mes INT,
            dia INT
        );
    '''))

    conn.execute(text('''
        CREATE TABLE fact_orden (
            id_detalle INT PRIMARY KEY,
            id_cliente INT REFERENCES dim_cliente(id_cliente),
            id_producto INT REFERENCES dim_producto(id_producto),
            id_tiempo INT REFERENCES dim_tiempo(id_tiempo),
            cantidad INT,
            precio_unitario NUMERIC(10, 2),
            subtotal NUMERIC(10, 2)
        );
    '''))

